In [12]:
import sys
import os
import shutil
# Get the absolute path to the project directory
project_dir = os.path.abspath("..")

# Append the project directory to sys.path
if project_dir not in sys.path:
    sys.path.append(project_dir)

import pandas as pd
import os
from src.common.AssetData import AssetData
from src.common.AssetDataService import AssetDataService
from src.common.AssetFileInOut import AssetFileInOut

# Create a temporary directory for file IO
directory = "./temp_test_dir"
os.makedirs(directory, exist_ok=True)
file_io = AssetFileInOut(directory)

In [ ]:
# Helper to build a sample AssetData
shareprice = pd.DataFrame([{
    'Date': '2025-04-20',
    'Open': 100.0,
    'High': 110.0,
    'Low': 90.0,
    'Close': 105.0,
    'AdjClose': 105.0,
    'Volume': 1000,
    'Dividends': 0.0,
    'Splits': 0.0
}])
financials_quarterly = pd.DataFrame([{
    'quarter': '2025-Q1',
    'reportedDate': '2025-04-15',
    'reportedEPS': 1.23,
    'estimatedEPS': 1.20,
    'surprise': 0.03,
    'surprisePercentage': 2.5,
    'reportTime': 'post-market',
    'grossProfit': 500000.0,
    'totalRevenue': 1000000.0,
    'ebit': 200000.0,
    'ebitda': 250000.0,
    'totalAssets': 5000000.0,
    'totalCurrentLiabilities': 1000000.0,
    'totalShareholderEquity': 4000000.0,
    'commonStockSharesOutstanding': 100000.0,
    'operatingCashflow': 150000.0,
    'profitLoss': 120000.0
}])
financials_annually = pd.DataFrame([{
    'year': '2024',
    'reportedEPS': 4.56,
    'grossProfit': 2000000.0,
    'totalRevenue': 4000000.0,
    'ebit': 800000.0,
    'ebitda': 1000000.0,
    'totalAssets': 20000000.0,
    'totalCurrentLiabilities': 4000000.0,
    'totalShareholderEquity': 16000000.0,
    'operatingCashflow': 600000.0,
    'profitLoss': 480000.0
}])

asset = AssetData(
    ticker="TEST",
    isin="US0000TEST",
    about={"name": "Test Asset"},
    sector="technology",
    shareprice=shareprice,
    financials_quarterly=financials_quarterly,
    financials_annually=financials_annually
)

# 6. Test empty DataFrames with correct columns
# Define expected columns
sp_cols = ['Date','Open','High','Low','Close','AdjClose','Volume','Dividends','Splits']
fq_cols = [
    'quarter','reportedDate','reportedEPS','estimatedEPS','surprise',
    'surprisePercentage','reportTime','grossProfit','totalRevenue','ebit',
    'ebitda','totalAssets','totalCurrentLiabilities','totalShareholderEquity',
    'commonStockSharesOutstanding','operatingCashflow','profitLoss'
]
fa_cols = [
    'year','reportedEPS','grossProfit','totalRevenue','ebit','ebitda',
    'totalAssets','totalCurrentLiabilities','totalShareholderEquity',
    'operatingCashflow','profitLoss'
]

empty_asset = AssetData(
    ticker="EMPTY",
    shareprice=pd.DataFrame(columns=sp_cols),
    financials_quarterly=pd.DataFrame(columns=fq_cols),
    financials_annually=pd.DataFrame(columns=fa_cols)
)

In [14]:
# 1. Test exists before save
assert not file_io.exists(asset.ticker), "File should not exist before saving"
print("✅ exists() before save works")

# 2. Test saveToFile and exists after save
file_io.saveToFile(asset)
assert file_io.exists(asset.ticker), "File should exist after save"
print("✅ saveToFile() and exists() after save work")

# 3. Test loadFromFile returns identical data
loaded = file_io.loadFromFile(asset.ticker)
orig_dict = AssetDataService.to_dict(asset)
loaded_dict = AssetDataService.to_dict(loaded)
assert orig_dict == loaded_dict, "Loaded data should match original"
print("✅ loadFromFile() returns correct AssetData")

# 4. Test saveDictToFile and loadDictFromFile
assets_dict = {"A": asset, "B": asset}
file_io.saveDictToFile(assets_dict, "assets_test")
assert file_io.exists("assets_test.pkl"), "Dict file should exist after save"
loaded_assets = file_io.loadDictFromFile("assets_test")
assert set(loaded_assets.keys()) == set(assets_dict.keys()), "Loaded dict keys should match"
for k in assets_dict:
    assert AssetDataService.to_dict(loaded_assets[k]) == AssetDataService.to_dict(assets_dict[k]), f"Asset {k} should match"
print("✅ saveDictToFile() and loadDictFromFile() work")

# 5. Test extension handling
ticker_ext = asset.ticker + ".pkl"
assert file_io.exists(ticker_ext), "exists() should handle .pkl extension"
print("✅ exists() handles .pkl extension correctly")

print("\nAll AssetFileInOut tests passed!")

✅ exists() before save works
✅ saveToFile() and exists() after save work
✅ loadFromFile() returns correct AssetData
✅ saveDictToFile() and loadDictFromFile() work
✅ exists() handles .pkl extension correctly

All AssetFileInOut tests passed!


In [15]:
# Save and load
file_io.saveToFile(empty_asset)
loaded_empty = file_io.loadFromFile("EMPTY")
# Check emptiness and column names
assert loaded_empty.shareprice.empty and list(loaded_empty.shareprice.columns) == sp_cols, "Empty shareprice df should preserve columns"
assert loaded_empty.financials_quarterly.empty and list(loaded_empty.financials_quarterly.columns) == fq_cols, "Empty quarterly df should preserve columns"
assert loaded_empty.financials_annually.empty and list(loaded_empty.financials_annually.columns) == fa_cols, "Empty annual df should preserve columns"
print("✅ Empty DataFrame tests passed")

print("\nAll AssetFileInOut tests passed!")

# Cleanup: remove test directory and all files
shutil.rmtree(directory)
print(f"🗑️ Removed directory '{directory}' and all contained files.")

✅ Empty DataFrame tests passed

All AssetFileInOut tests passed!
🗑️ Removed directory './temp_test_dir' and all contained files.
